In [ ]:
def get_distance(location1, location2):
    x1, y1 = map(int, location1.split(','))
    x2, y2 = map(int, location2.split(','))
    return abs(x1 - x2) + abs(y1 - y2)
    

In [1]:
ia = {'jhi':1,
 'co':{'that':1, 'this':2}}

In [2]:
ia['co'].pop('that')


1

In [3]:
ia

{'co': {'that': 1, 'this': 2}}